## CAS Deep Learning - Computer Vision mit Deep Learning (Part 1)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Image Classification

Diese Übung basiert zum Teil auf: [https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)


## Lernziele

- CNNs: Definieren, Optimieren, Inspizieren
- Klassifikation: Definieren, Evaluieren

## Setup

Im Folgenden installieren und laden wir die benötigten Python packages. Danach setzten wir die Pfade für den Zugriff auf Daten und spezifizieren einen Output-Folder.

In [ ]:
import os
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
from tqdm.notebook import tqdm

Mount your google drive to store data and results.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print(f"In colab: {IN_COLAB}")

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Modifizieren Sie die folgenden Pfade bei Bedarf.

In [ ]:
if IN_COLAB:
    DATA_PATH = Path('/content/drive/MyDrive/bverI/data')
else:
    DATA_PATH = Path('../data')

Install packages not in base Colab environment.

In [ ]:
if IN_COLAB:
    os.system("pip install torchshow torchinfo gdown")

import torchshow as ts
import torchinfo

## Pre-Trained Models


### Modell Laden

Wir verwenden ein vortrainiertes Modell (Lesen Sie dazu die Dokumentation: [https://pytorch.org/vision/0.8/models.html](https://pytorch.org/vision/0.8/models.html).)

Ändern Sie den folgenden Code, sodass Sie `ResNet18` mit vortrainierten Gewichten laden, anstatt `Alexnet`.

In [ ]:
import torchvision.models as models

net = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
classes = models.AlexNet_Weights.DEFAULT.meta["categories"]

# YOUR CODE HERE
raise NotImplementedError()

print(torchinfo.summary(net, input_size=(1, 3, 224, 224)))

Visualisieren Sie die gelernten Filter im ersten Convolutional Layer. Mit `list(net.parameters())` erhalten Sie eine Liste mit allen trainierbaren Modell-Parametern. Selektieren Sie die Gewichte des ersten Convolutional Layers. Visualisieren Sie die gelernten Gewichte (die Filters) mit `torchshow.show()`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**FRAGE**: Was sehen Sie? Wie interpretieren Sie die Filters?

**FRAGE:** Können Sie auch die Filter im darauf folgenden Convolutional Layer visualisieren? Warum?

### Modell Anwenden

Wir laden nun Bilder von Katzen und Hunden.

In [ ]:
import gdown

file_id = "1WLO1LOwIp82ZTyf3eKjRAo65FBMaAfuo"
url = f"https://drive.google.com/uc?id={file_id}"

download_path = DATA_PATH / "cats_vs_dogs.zip"
if not download_path.exists():
    gdown.download(url, str(download_path), quiet=False)


In [ ]:
if not (DATA_PATH / "cats_vs_dogs").exists():
    CMD = f"unzip {str(download_path)} -d {DATA_PATH}"
    os.system(CMD)

Lesen Sie die Bilder:
- `DATA_PATH / "cats_vs_dogs" / "dog.4.jpg`
- `DATA_PATH / "cats_vs_dogs" / "cat.38.jpg`

Sie können `PIL.Image.open()` verwenden.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

display(cat)

display(dog)

Erzeugen Sie _Predictions_ für die beiden Bilder. **Achtung**: Sie müssen die Dokumentation in [torchvision](https://pytorch.org/vision/0.8/models.html) genau lesen. Sind Sie zufrieden?

In [ ]:
from torchvision import transforms
import torch.nn.functional as F


for img in [cat, dog]:

    x = transforms.functional.pil_to_tensor(img) / 255.0

    # Something import needs to happen here: read the docs
    # YOUR CODE HERE
    raise NotImplementedError()

    # we add a batch dimension, which is expected by the model
    x = x.unsqueeze(0).float()

    # we apply a forward pass
    with torch.no_grad():
        _ = net.eval()
        out = net(x)
        
        # select the most likely prediction / class
        # prediction = ...
        # YOUR CODE HERE
        raise NotImplementedError()

    print(f"Prediction: {classes[prediction]}")


### Inspektion von Activation Maps

Wir schauen uns nun die Activation Maps an, die während dem Forward-Pass erstellt werden. Wir benutzen dazu einen _hook_: https://web.stanford.edu/~nanbhas/blog/forward-hooks-pytorch/

Sie können diesen folgendermassen registrieren:

```
activation = {}
def getActivation(name):
  # the hook signature
  def hook(model, input, output):
    activation[name] = output.detach()
  return hook
  
 hook = list(net.modules())[30].register_forward_hook(getActivation('conv'))
```

Visualisieren Sie die _Activation Maps_ mit den grössten _activations_.list(net.modules()). Benutzen Sie wiederum `torchvision.show` um die Activation Maps darzustellen. Vergleichen Sie mit dem Bild.

In [ ]:
activation = {}
def getActivation(name):
  # the hook signature
  def hook(model, input, output):
    activation[name] = output.detach()
  return hook

# we register the hook at a specific position in the model
hook = list(net.modules())[30].register_forward_hook(getActivation('conv'))

# now we perform a forward pass (x is one of the images)
with torch.no_grad():
    _ = net.eval()
    out = net(x)

idx_sorted_activations = activation['conv'].abs().sum(dim=(0, 2, 3)).argsort()
activations_to_plot = activation['conv'].squeeze(0)[idx_sorted_activations[-16:], : ,: ]
activations_scaled = (activations_to_plot - activations_to_plot.min()) / (activations_to_plot.max() - activations_to_plot.min())

# Display the activation maps
# YOUR CODE HERE
raise NotImplementedError()

## Definieren, Trainieren und Evaluieren eines CNN für ein Klassifikationsproblem


### Datensatz

Als erstes bereiten wir den CIFAR10 Datensatz vor.

In [ ]:
import torchvision
import torchvision.transforms as transforms

Benutzen Sie `torchvision.datasets.CIFAR10` um zwei `torch.utils.data.Dataset` zu erstellen. 

- Eines für den Trainings-Datensatz
- Eines für den Test-Datensatz

Instanzieren Sie dann jweils einen`torch.utils.data.Dataloader` um über die Daten zu iterieren.

Erstellen Sie `torchvision.transforms` Transformationen um die Daten in Tensoren `transforms.ToTensor()` zu konvertieren und die Daten so mit `transforms.Normalize` zu skalieren, dass diese im Interval [-1, 1] liegen.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


Benutzen Sie `torchshow.show()` um die Bilder eines Batches anzuzeigen. Plotten Sie zusätzlich die Labels.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


### Definition CNN

Definieren Sie ein CNN mit folgender Sequenz:

- Input Shape: (3, 32, 32)
- Convolution: 16 Filters, Kernel-Size 5x5
- Pooling: Stride 2, Kernel-Size 2
- Convolution: 32 Filter, Kernel-Size 5x5
- Global Average Pooling
- FC: 10 Neuronen (für 10 Klassen)

Benutzen Sie ReLU Aktivierungen nach jeder Convolution

Definieren Sie eine Klasse, die von `torch.nn.Module` erbt und instanzieren Sie Ihr Netzwerk.

In [ ]:
import torchinfo
import torch.nn as nn
import torch.nn.functional as F

# YOUR CODE HERE
raise NotImplementedError()



**Frage:** Was ist der Vorteil von Global Average Pooling? Die Alternative wären weitere Fully-Connected Layers. 

### Trainieren eines CNN

Erstellen Sie eine Objekt für die [_cross entropy_](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) Kostenfunktion und instanzieren Sie einen _optimizer_ aus [torch.optim](https://pytorch.org/docs/stable/optim.html).

In [ ]:
import torch.optim as optim
# YOUR CODE HERE
raise NotImplementedError()

Erstellen Sie einen Trainings-Loop und trainieren Sie ihr Modell für 2 Epochen. Printen Sie den Loss in regelmässigen Abständen.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Speichern Sie ihr Modell ab.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Visualisieren Sie die gelernten _filters_ im ersten Layer des CNNs. Sie können diese via `net.parameters()` finden. Benutzen Sie `torchshow.show()` für die Visualisierung.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Evaluieren eines CNN

Wir möchten unser Modell nun auf dem Test-Set Evaluieren. Als erstes erzeugen wir _predictions_ für einen Batch von Test-Bildern:

- Benutzen Sie den dataloader um 16 Bilder zu laden.

- Erstellen Sie Predictions für die 16 Bilder mit einem forward-pass durch das Modell.

- Vergleichen Sie die Predictions mit dem wahren Label.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Evaluieren Sie nun den gesamten Test-Datensatz. Benutzen Sie `torch.no_grad()` um keinen Graphen aufzubauen. Berechnen Sie die Accuracy und printen Sie diese.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Confusion-Matrix

Plotten Sie eine _confusion matrix_. Benutzen Sie 

- [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
- [ConfusionMatrixDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# YOUR CODE HERE
raise NotImplementedError()

**Frage:** Welche Klassen werden stark verwechselt? Warum wohl?